# Basis Change of the System (Spin operators and States)
- Author: Vineeth Francis Thalakottoor
- Email: vineeth.thalakottoor@ens.psl.eu or vineethfrancis.physics@gmail.com

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/PyOR/Source_Doc'

# Add source path
import sys
sys.path.append(SourcePath)
import time
%matplotlib ipympl

# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_DensityMatrix import DensityMatrix
from PyOR_QuantumObject import QunObj
from PyOR_HardPulse import HardPulse
from PyOR_Basis import Basis
from PyOR_Evolution import Evolutions
from PyOR_Plotting import Plotting
import PyOR_SignalProcessing as Spro
from PyOR_Commutators import Commutators
from PyOR_QuantumLibrary import QuantumLibrary
from PyOR_Relaxation import RelaxationProcess

In [2]:
# Define the spin system
Spin_list = {"A" : "H1", "B" : "H1"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Spin Operators
- After (first) QS.Initialize(), spin operators are always in Zeeman basis

In [3]:
QS.Ax.matrix

Matrix([
[  0,   0, 0.5,   0],
[  0,   0,   0, 0.5],
[0.5,   0,   0,   0],
[  0, 0.5,   0,   0]])

### Set parameters

In [4]:
# Master Equation
QS.PropagationSpace = "Hilbert"
QS.MasterEquation = "Redfield"

# Operator Basis
#QS.Basis_SpinOperators_Hilbert = "Singlet Triplet" # or
QS.Basis_SpinOperators_Hilbert = "Zeeman"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = 9.4

# Offset Frequency in rotating frame (Hz)
QS.OFFSET["A"] = 10.0
QS.OFFSET["B"] = 50.0

# Define J coupling between Spins 
QS.JcoupleValue("A","B",5.0)

# Define initial and final Spin Temperature
QS.I_spintemp["A"] = 300.0
QS.I_spintemp["B"] = 300.0
QS.F_spintemp["A"] = 300.0
QS.F_spintemp["B"] = 300.0

# Define paris of spins coupled by dipolar interaction
QS.Dipole_Pairs = [("A","B")]

# Relaxation Process
QS.Rprocess = "Auto-correlated Dipolar Homonuclear"
QS.RelaxParDipole_tau = 10.0e-12
QS.RelaxParDipole_bIS = [30.0e3]

QS.Update()

Larmor Frequency in MHz:  [-400.22802765 -400.22806765]


### Generate Hamiltonians

In [5]:
# generate Larmor Frequencies

Ham = Hamiltonian(QS)
COMM = Commutators(QS)

# Zeeman Hamiltonian in rotating frame
Hz = Ham.Zeeman_RotFrame()

# J coupling Hamiltonian
Hj = Ham.Jcoupling()

# Generating the commutation superoperator
QLib = QuantumLibrary(QS)

Larmor Frequency in MHz:  [-400.22802765 -400.22806765]


In [6]:
Hz.Inverse2PI().matrix

Matrix([
[-30.0000000759161,                0,                 0,                0],
[                0, 20.0000008601147,                 0,                0],
[                0,                0, -20.0000008601147,                0],
[                0,                0,                 0, 30.0000000759161]])

In [7]:
Hj.Inverse2PI().Tolarence(1.0e-5).matrix

Matrix([
[1.24999997786618,                 0,                 0,                0],
[               0, -1.24999997786618,  2.49999995573235,                0],
[               0,  2.49999995573235, -1.24999997786618,                0],
[               0,                 0,                 0, 1.24999997786618]])

### Calling Hamiltonian class from QunautmSystem class
- Alternate way of calling Hamiltonian class from QuantuSystem class

In [8]:
Hj1 = QS.Class_hamiltonian.Jcoupling()
Hj1.Inverse2PI().Tolarence(1.0e-5).matrix

Matrix([
[1.24999997786618,                 0,                 0,                0],
[               0, -1.24999997786618,  2.49999995573235,                0],
[               0,  2.49999995573235, -1.24999997786618,                0],
[               0,                 0,                 0, 1.24999997786618]])

### Spin operators
- After QS.Update, all the spin operators will be in the new basis

In [9]:
QS.Ax.matrix

Matrix([
[  0,   0, 0.5,   0],
[  0,   0,   0, 0.5],
[0.5,   0,   0,   0],
[  0, 0.5,   0,   0]])

### Basis states
- Singlet and triplet
- Zeeman

In [10]:
# Singlet Triplet states
ST_B, dic_ST = QS.Class_basis.SingletTriplet_Basis()

# Zeeman states
Z_B, dic_Z = QS.Class_basis.Zeeman_Basis()

In [11]:
# Singlet triplet states dictionary
dic_ST

['S0 ', 'Tp ', 'T0 ', 'Tm ']

In [12]:
# Zeeman states dictionary
dic_Z

['|1/2,1/2⟩|1/2,1/2⟩',
 '|1/2,1/2⟩|1/2,-1/2⟩',
 '|1/2,-1/2⟩|1/2,1/2⟩',
 '|1/2,-1/2⟩|1/2,-1/2⟩']

In [13]:
ST_B[0].matrix

Matrix([
[                 0],
[ 0.707106781186547],
[-0.707106781186547],
[                 0]])

In [14]:
Z_B[1].matrix

Matrix([
[  0],
[1.0],
[  0],
[  0]])

In [15]:
# Decompose the states in basis states
QS.Class_basis.KetState_Components(Z_B[3],ST_B, dic_ST)

Ket State = 1.0 Tm 


In [32]:
# Create a stat with magnetic quantum number
X1 = QS.State({"A": -1/2,"B": -1/2})
X1.matrix

Matrix([
[  0],
[  0],
[  0],
[1.0]])

In [33]:
QS.Class_basis.KetState_Components(X1, ST_B, dic_ST)

Ket State = 1.0 Tm 


In [34]:
QS.Class_basis.KetState_Components(X1, Z_B, dic_Z)

Ket State = 1.0 |1/2,-1/2⟩|1/2,-1/2⟩


In [35]:
QS.Class_basis.KetState_Components(ST_B[0], Z_B, dic_Z)

Ket State = 0.70711 |1/2,1/2⟩|1/2,-1/2⟩ + -0.70711 |1/2,-1/2⟩|1/2,1/2⟩
